In [2]:
import os
import sys
import json
from pathlib import Path

import torch
import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset
from jupyter_client import ioloop
%config ServerApp.iopub_data_rate_limit = 1.0e20

In [5]:
data_dict_train_input = {}
data_dict_train_output = {}
data_list_train_input = []
data_list_train_output = []
temp_data_list = []
maxlength = 0
maxwidth = 0

def training_handling_challenges(data_folder, data_dict_train_input):
    
    file_path = Path(data_folder).expanduser()
    with file_path.open("r") as f:
        file_dict = json.load(f)

    for data in file_dict:
        train_data_list = []
        train_data = file_dict[data]['train']
        for train_item in train_data:
            train_data_list.append(train_item['input'])

        #for item in train_data_list:
            #print(len(item))
        data_dict_train_input[data] = train_data_list
        #print('seperate')


def training_handling_solutions(data_folder, data_dict_train_output):
    file_path = Path(data_folder).expanduser()
    with file_path.open("r") as f:
        file_dict = json.load(f)

    for data in file_dict:
        train_data_list = []
        train_data = file_dict[data]['train']
        for train_item in train_data:
            train_data_list.append(train_item['output'])

        #for item in train_data_list:
            #print(len(item))
        data_dict_train_output[data] = train_data_list
        #print('seperate')


def transform_and_pad(data_values):
    for i, item in enumerate(data_values):
        for j, item2 in enumerate(item):
            for k, item3 in enumerate(item2):
                for l, item4 in enumerate(item3):
                    data_values[i][j][k][l] += 1
                    data_values[i][j][k][l] = data_values[i][j][k][l] / 10.0 
                
                
                
    #Pad data  
    for i, item in enumerate(data_values):
        for j, item2 in enumerate(item):
            width = len(data_values[i][j][0])
            length = len(data_values[i][j])
            for k, item3 in enumerate(item2):
                missing_w = 30 - width
                data_values[i][j][k].extend([0] * missing_w)
                #print(data_values[i][j][k])
            missing_l = 30 - length
            if missing_l > 0:
                zeros_rows = [[0] * 30 for _ in range(missing_l)]
                data_values[i][j].extend(zeros_rows)
            #print(f"After padding: {len(data_values[i][j])}x{len(data_values[i][j][0])}")
            #print()
            
    return data_values

    


training_handling_challenges(f"~/ARC-AGI-Model/src/data_pipeline/arc-data-cleaned/arc-agi_training_challenges.json", data_dict_train_input)
training_handling_solutions(f"~/ARC-AGI-Model/src/data_pipeline/arc-data-cleaned/arc-agi_training_challenges.json", data_dict_train_output)


#for item in data_dict_train_output.values():
    #temp_data_list.append(item)

#for item in temp_data_list:
    #for item2 in item:

        #temp_item = np.array(item2)
        #shape = np.shape(temp_item)
        #if (shape[0] > maxlength):
            #maxlength = shape[0]
        #if (shape[1] > maxwidth):
            #maxwidth = shape[1]
        

#print(maxlength)
#print(maxwidth)

#Creates a list of each training example (variable list length)
dict_length = 0
for item in data_dict_train_input.keys():
    if (len(data_dict_train_input[item]) > dict_length):
        dict_length = len(data_dict_train_input[item])
print(dict_length)

for item in data_dict_train_input.values():
        data_list_train_input.append(item)

for item in data_dict_train_output.values():
        data_list_train_output.append(item)

train_input_transformed = transform_and_pad(list(data_dict_train_input.values()))
train_output_transformed = transform_and_pad(list(data_dict_train_output.values()))



print(train_output_transformed)

class TrainData(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
        
    def __len__(self):
        return 30



8
[[[[0.1, 0.1, 0.1, 0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.1, 0.1, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.1, 0.1, 0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.1, 0.1, 0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.1, 0.1, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.1, 0.1, 0.1, 0.1, 0.8, 0.8, 0.1, 0.8, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0